# TF Logical Gates Demo
Train overfitted networks for logical gates (AND, NOR, OR, XOR. XNOR).

In [1]:
import tensorflow as tf
from tensorflow import keras
from tensorflow.keras import layers

tf.__version__

'2.11.0'

### Training data

In [2]:
# All possible x-input combinations.
x_train = tf.constant([[0,0], [0,1], [1,0], [1,1]], dtype=tf.float32)

# y-outputs for some gates.
y_and  = tf.constant([[0],[0],[0],[1]], dtype=tf.float32)
y_nor  = tf.constant([[1],[1],[1],[0]], dtype=tf.float32)
y_or   = tf.constant([[0],[1],[1],[1]], dtype=tf.float32)
y_xor  = tf.constant([[0],[1],[1],[0]], dtype=tf.float32)
y_xnor = tf.constant([[1],[0],[0],[1]], dtype=tf.float32)

input_shape = 2
num_classes = 2;

### Helper functions

In [3]:
def predict_element(model, x_element, y_ref_element):
    x_element = tf.expand_dims(x_element, 0)
    pred = model.predict(x_element, verbose=0)
    # Determine zero or 1.
    if pred < 0.5:
        pred = 0
    else:
        pred = 1
    # Match with reference.
    ret = True
    if pred != y_ref_element:
        ret = False
    print(f"|{x_element[0]}|{y_ref_element}| -> {pred} -> {ret}")
    return ret

In [4]:
def predict_gate(model, x_gate, y_gate):
    pred_ok = 0
    for x_in,y_ref in zip(x_gate, y_gate):
        pred = predict_element(model, x_in, y_ref)
        if pred == True:
            pred_ok += 1
    print(f"----> {(pred_ok/len(x_gate))*100}%")

In [5]:
def save_tf_model(model, name, save_format="tf"):
    if save_format == "tf" or save_format == "all":
        # Save model in TensorFlow SavedModel format
        model.save(f'{name}')
    
    elif save_format == "h5" or save_format == "all":
        # Save model and weights in h5 format.
        model.save(f'{name}.h5', save_format='h5')
    
    elif save_format == "h5_weights" or save_format == "all":
        # Save weights only in h5 format.
        model.save_weights(f'{name}_weights.h5')
    elif save_format == "json" or save_format == "all":
        # Save model architecture as json
        json = model.to_json()
        json_file = open(f"{name}_architecture.json", "w")
        json_file.write(json)
        json_file.close()
    else:
        raise "Unsupported save_format!"


### Build model

In [6]:
def build_tf_model(input_shape):
    model = keras.Sequential()
    model.add(keras.layers.Input(shape=input_shape))
    model.add(keras.layers.Dense(4, activation='sigmoid'))
    model.add(keras.layers.Dense(1, activation='sigmoid'))
    return model

### Train a model

In [7]:
def build_and_train_tf_model(x_gate, y_gate, name, additional_prints=False, epochs=5000, save_model=False):
    print(f"Training {name}...")
    
    # Build model.
    input_shape = len(x_gate[0])
    model = build_tf_model(input_shape)
    
    if additional_prints:
        model.summary()
    
    # Compile and train model.
    model.compile(loss="mean_squared_error", optimizer="adam", metrics=["accuracy"])
    model.fit(x_gate, y_gate, batch_size=1, epochs=epochs, verbose=0)
    
    # TF standard evaluate.
    if additional_prints:
        print("TF evaluate model")
        results = model.evaluate(x_gate, y_gate, batch_size=1)
        print("test loss, test acc:", results)
    
    # Custom evaluate.
    predict_gate(model, x_gate, y_gate)
    
    if save_model:
        save_tf_model(model, name)
    
    return model


### Train and verify models

In [8]:
build_and_train_tf_model(x_train, y_and, "and-gate", save_model=True)
build_and_train_tf_model(x_train, y_nor, "nor-gate", save_model=True)
build_and_train_tf_model(x_train, y_or, "or-gate", save_model=True)
build_and_train_tf_model(x_train, y_xor, "xor-gate", epochs=15000, save_model=True)
build_and_train_tf_model(x_train, y_xnor, "xnor-gate", epochs=15000, save_model=True)

Training and-gate...
|[0. 0.]|[0.]| -> 0 -> True
|[0. 1.]|[0.]| -> 0 -> True
|[1. 0.]|[0.]| -> 0 -> True
|[1. 1.]|[1.]| -> 1 -> True
----> 100.0%


INFO:tensorflow:Assets written to: and-gate\assets


INFO:tensorflow:Assets written to: and-gate\assets


Training nor-gate...
|[0. 0.]|[1.]| -> 1 -> True
|[0. 1.]|[1.]| -> 1 -> True
|[1. 0.]|[1.]| -> 1 -> True
|[1. 1.]|[0.]| -> 0 -> True
----> 100.0%


INFO:tensorflow:Assets written to: nor-gate\assets


...

INFO:tensorflow:Assets written to: xor-gate\assets


INFO:tensorflow:Assets written to: xor-gate\assets


Training xnor-gate...
|[0. 0.]|[1.]| -> 1 -> True
|[0. 1.]|[0.]| -> 0 -> True
|[1. 0.]|[0.]| -> 0 -> True
|[1. 1.]|[1.]| -> 1 -> True
----> 100.0%


INFO:tensorflow:Assets written to: xnor-gate\assets


INFO:tensorflow:Assets written to: xnor-gate\assets
